In [3]:
import numpy as np
from dataset import SP_N as sp
from dataset import SP_rh as sp2
import matplotlib.pyplot as plt
from portfolio import Portfolio as pf
import estimation as est
import trade

pi1 = trade.dailytrade(60, 5, 10, 1000)

Day 0 - Optimization
Gamma - 300


Ticker - AMZN, # Stocks - 1.0


In [2]:
n = 1
d2 = sp2(n)
na = 5
nb = 10
ndays = d2.days()
a = d2.getall()
"""a = np.zeros((n, ndays, 5))
for k in range(n):
    a[k] = d2[k]
print(a.shape)"""

def split(data):
    splitv = .8
    sint = int(splitv * ndays)
    train_d = data[:, :sint]
    test_d = data[:, sint - 10:]
    return train_d, test_d

train_d, test_d = split(a)
print(train_d.shape)
print(test_d.shape)

(1, 1006, 5)
(1, 262, 5)


In [5]:
thta = []
output = []
rho = .98
c = .4

mout = np.zeros((n, na, test_d.shape[1] - na - nb + 1))
for k in range(n):
    ph1, input2, output2 = est.makearmodel(a[k], na, nb, rho, c, "rls", False)
    thta.append(ph1)
    mout[k], ph2 = est.priceest(test_d[k], thta[k], na, nb)
    output.append(ph2)

In [18]:
print(a[0, -15:-5, :-1].flatten())
print(input2.shape)
print(input2[-1])

[173.62   172.57   174.06   171.     173.16   172.07   173.21   171.47
 171.99   172.07   173.1383 171.7991 171.71   172.69   172.925  170.4201
 173.     175.05   175.24   172.58   176.39   175.16   176.39   174.94
 173.98   174.2    174.71   173.45   173.13   171.56   173.3794 171.275
 171.09   171.84   172.4183 170.52   172.41   172.99   173.895  171.69  ]
(1244, 40)
[173.62   172.57   174.06   171.     173.16   172.07   173.21   171.47
 171.99   172.07   173.1383 171.7991 171.71   172.69   172.925  170.4201
 173.     175.05   175.24   172.58   176.39   175.16   176.39   174.94
 173.98   174.2    174.71   173.45   173.13   171.56   173.3794 171.275
 171.09   171.84   172.4183 170.52   172.41   172.99   173.895  171.69  ]


In [24]:
din, dout = est.getdata(train_d[0], 5, 10)
print(din.shape)
print(dout.shape)

print(mout.shape)
print(test_d.shape)

def returns(data):
    ret = np.zeros((data.shape[0], data.shape[1] - 1))
    for k in range(data.shape[0]):
        ret[k] = data[k, 1:] / data[k, :-1]
    return ret
cov = np.cov(returns(train_d[:, -41:, 3]))
print(cov.shape)

(992, 40)
(992, 5)
(40, 5, 248)
(40, 262, 5)
(40, 40)


In [28]:
initv = 2000
days = 248

In [29]:
# Single Int
pi = pf(initv, days + 10, 1, n, cov, .00001, train_d[:, -1, 3], 300, False)
returnsi = np.zeros((days,))
dvali = np.zeros((days,))
for k in range(days):
    pi.trade(mout[:, 0, k], False, True)
    dvali[k], returnsi[k] = pi.analyze(test_d[:, k, 3])

Day 0 - Optimization
Gamma - 300


Day 1 - Analysis
Estimate Value at t=1 - 2005.2056941760236
Value at  - 1998.1960824000003
Daily Return - -0.0901958799999818%
Total Return - -0.0901958799999818%
Max Daily Return - 7.695744443203396%
Gross Exposure - 399.94010000000003
Leverage at - 0.20015057757476865
Turnover at - 0.10044


Day 1 - Optimization
Gamma - 300


Day 2 - Analysis
Estimate Value at t=2 - 2199.7387249765075
Value at  - 1990.3480569858245
Daily Return - -0.392755520006304%
Total Return - -0.4825971507087701%
Max Daily Return - 1.344298547443435%
Gross Exposure - 1623.88581
Leverage at - 0.8158803201783744
Turnover at - 0.5083779359530586


Day 2 - Optimization
Gamma - 300


Day 3 - Analysis
Estimate Value at t=3 - 2173.890797961812
Value at  - 1993.5116831663943
Daily Return - 0.15894838942696765%
Total Return - -0.3244158416802776%
Max Daily Return - 3.10546481145392%
Gross Exposure - 1630.8899999999999
Leverage at - 0.8180990428957887
Turnover at - 0.5873987219956507


D

In [21]:
# Single
p = pf(initv, days + 10, 1, n, cov, .00001, train_d[:, -1, 3], 300, False)
returns = np.zeros((days,))
dval = np.zeros((days,))
for k in range(days):
    p.trade(mout[:, 0, k], False, False)
    dval[k], returns[k] = p.analyze(test_d[:, k, 3])

Day 0 - Optimization
Gamma - 300


Day 1 - Analysis
Estimate Value at t=1 - 3016.558501537247
Value at  - 2999.94639430794
Daily Return - -0.001786856402006265%
Total Return - -0.001786856402006265%
Max Daily Return - 2.947255748315468%
Gross Exposure - 1025.087191173738
Leverage at - 0.3417018361123803
Turnover at - 0.1708600908767241


Day 1 - Optimization
Gamma - 300


Day 2 - Analysis
Estimate Value at t=2 - 3379.372894817993
Value at  - 3062.3095594111364
Daily Return - 2.078809315443891%
Total Return - 2.0769853137045544%
Max Daily Return - 6.699485856308156%
Gross Exposure - 3062.309558937872
Leverage at - 0.9999999998454552
Turnover at - 0.6453836234704622


Day 2 - Optimization
Gamma - 300


Day 3 - Analysis
Estimate Value at t=3 - 3392.3830530006844
Value at  - 3077.3439083092117
Daily Return - 0.49094804448725426%
Total Return - 2.5781302769737238%
Max Daily Return - 2.0968457701801144%
Gross Exposure - 2951.141504212573
Leverage at - 0.9589898276380887
Turnover at - 0.32138

In [ ]:
# Single Oracle
po = pf(initv, days + 10, 1, n, cov, .00001, train_d[:, -1, 3], 300, False)
returns_o = np.zeros((days,))
dval_o = np.zeros((days,))
for k in range(days):
    po.trade(test_d[:, k, 3], False, True)
    dval_o[k], returns_o[k] = po.analyze(test_d[:, k, 3])

In [ ]:
# Mulit Int
pm = pf(initv, days + 10, 5, n, cov, .00001, train_d[:, -1, 3], .02, False)
returns_m = np.zeros((days,))
dval_m = np.zeros((days,))
for k in range(days):
    pm.trade(mout[:, :, k].T, False, True, .2)
    dval_m[k], returns_m[k] = pm.analyze(test_d[:, k, 3])